# Sample Skyspark VAV Validation

# 1) Setup

## Imports

In [ ]:
# ----------------------------------------
# Imports
# ----------------------------------------
import os
import json
import re

from rdflib import Namespace, SH, RDF, BNode, Graph
from pyshacl import validate

from tasty import constants as tc
from tasty import graphs as tg
from tasty.skyspark import client as cl
from tasty.skyspark import process_graphs as pg
from tasty.skyspark import helpers

## Inputs
Define the key variables and input information here

***Items to Change***
- `SHAPE`: this is the name of the SHACL equipment shape against which you would like to validate your sample equipment in the instance data
- `SAMPLE`: this is the name of the sample equipment in your instance data
- `input_namespace_uri`: this is the namespace uri used for your sample equipment in the instance data
- `raw_data_graph_filename`: this is the filename/filepath to save the raw instance data (in turtle format) retrieved from the Skyspark API call
- `data_graph_filename`: this is the filename/filepath to save the cleaned/processed instance data for the data graph to be used for validation
- `shapes_graph_filename`: this it the filename/filepath of the SHACL shapes data for the shape graph 
***Remaining Items*** </br>
These items should be okay as is, but can be changed if need be. If you are printing out results, <u>*make sure that the output directory exists in your local file structure*</u>.
- `output_directory`: this is the directory where output files will be printed to below
- `tasty_main_directory`: this is the absolute path of the main tasty directory. It should just be the parent directory of the current working directory.

In [ ]:
# ----------------------------------------
# User Defined Variables
# ----------------------------------------

equip_name = 'AHU1'
building = 'TTF'

SHAPE = 'NREL-AHU-VAV-MZ-HW-Evap-Shape'
SAMPLE = '' # will be populated below
input_namespace_uri = 'urn:/_#'

raw_data_graph_filename = 'examples/output/sample_skyspark_vav_raw.ttl'
data_graph_filename = 'examples/output/sample_skyspark_vav_clean.ttl'
shapes_graph_filename = 'tasty/generated_shapes/haystack_all.ttl'

output_directory = os.path.join(os.path.abspath(''), 'example_data/output')
tasty_main_directory = os.path.join(os.path.abspath(''), '../')
# print(tasty_main_directory)

# ----------------------------------------
# Variables and Constants
# ----------------------------------------

NAMESPACE = Namespace(input_namespace_uri)
shape_name = tc.PH_SHAPES_NREL[SHAPE]
target_node = NAMESPACE[SAMPLE]

## API Request From Skyspark 
NOTE - Must be connected to NREL network to access the api endpoint

In [ ]:
# axon_query_string = '(point and equipRef->navName=="UFVAV-3") or (equip and navName=="UFVAV-3")'
api_url_endpoint = 'https://internal-apis.nrel.gov/intelligentcampus/stm_campus/read'

client = cl.SkysparkClient(api_url_endpoint)
axon_query_string = client.generate_axon_query_for_equip(equip_name, building)
print(axon_query_string)

response = client.make_get_request(axon_query_string, 'turtle')

print(response.status_code, end = " - ")
if response.status_code == 200:
    print("Sucess")
elif response.status_code == 404:
    print("Not Found")

raw_skyspark_data = response.text
# print(raw_skyspark_data)

In [ ]:
# Save response to file
f = os.path.join(tasty_main_directory, raw_data_graph_filename)

helpers.save_data_to_file(raw_skyspark_data, f)

print(f"raw instance data saved to '{raw_data_graph_filename}' ")

#### Get the ID of the Equipment

In [ ]:
SAMPLE = client.get_equip_id(equip_name, building)
print(SAMPLE)

# 2) Generate Graphs

### Pre-Process Raw Input File

In [ ]:
# ----------------------------------------
# Create instance of SkysparkGraphProcessor
# ----------------------------------------

schema = tc.HAYSTACK
version = tc.V3_9_10

sgp = pg.SkysparkGraphProcessor(input_namespace_uri,schema, version)

# ----------------------------------------
# Pre Process raw skyspark .ttl file 
# ----------------------------------------
f1 = os.path.join(tasty_main_directory, raw_data_graph_filename)
f2 = os.path.join(tasty_main_directory, data_graph_filename)

sgp.clean_raw_skyspark_turtle(f1,f2)

print(f"cleaned instance data saved to '{data_graph_filename}' ")

### Create Data, Shapes, and Ontology Graphs

In [ ]:
# ----------------------------------------
# Generate Graphs
# ----------------------------------------

# Data Graph
dg_file = os.path.join(tasty_main_directory, data_graph_filename)
data_graph = sgp.get_data_graph(dg_file)
print("...loaded data graph")

# Shapes Graph
sg_file = os.path.join(tasty_main_directory, shapes_graph_filename)
shapes_graph = sgp.get_shapes_graph(sg_file, target_node, shape_name)
print("...loaded shapes graph")

# Ontology Graph
ont_graph = sgp.get_ontology_graph()
print("...loaded ontology graph")

In [ ]:
helpers.print_graph(data_graph)

# 3) Validation

## PySHACL Validation

In [ ]:
# ----------------------------------------
# Run pySCHACL Validation
# ----------------------------------------
result = validate(data_graph, shacl_graph=shapes_graph, ont_graph=ont_graph)
conforms, results_graph, results = result

print(f"Conforms: {conforms}")

In [ ]:
helpers.print_graph(results_graph)

## Determine Missing Points

In [ ]:
missing_points = sgp.determine_missing_points(results_graph)

if len(missing_points['required']) <= 0:
    print("No Required Points Missing")
else:
    print(f"{len(missing_points['required'])} Missing Required Points:")
    for point in missing_points['required']:
        print(f"\t{point}")
        
if len(missing_points['optional']) <= 0:
    print("No Optional Points Missing")
else:
    print(f"{len(missing_points['optional'])} Missing Optional Points:")
    for point in missing_points['optional']:
        print(f"\t{point}")
            